In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import os
from tqdm import tqdm

print("Setup complete")


Setup complete


In [4]:
# Define transformations for training and validation
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # Add color jitter
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762))
])

# Download CIFAR-100 dataset
train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

print("Data loaded")


100%|██████████| 169001437/169001437 [02:50<00:00, 989129.88it/s] 


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified
Data loaded


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [7]:
# Load pretrained ResNet101 and modify for CIFAR-100 classes (100)
model = torchvision.models.resnet101(pretrained=True)

# Change first conv layer to kernel=3, stride=1 to accommodate 32x32 CIFAR images
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()  # Remove maxpool to keep feature map size

model.fc = nn.Linear(model.fc.in_features, 100)  # CIFAR-100 classes

model = model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=1e-6)


print(f"Model ready on {device}")


/home/shree/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/shree/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model ready on cuda


In [8]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    return running_loss / len(loader.dataset)

def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

num_epochs = 100
for epoch in range(num_epochs):
    loss = train_epoch(model, train_loader, criterion, optimizer, device)
    accuracy = evaluate(model, test_loader, device)
    scheduler.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss:.4f}, Test Accuracy: {accuracy*100:.2f}%")


100%|██████████| 391/391 [03:03<00:00,  2.13it/s]


Epoch [1/100], Loss: 1.9476, Test Accuracy: 64.82%


100%|██████████| 391/391 [03:04<00:00,  2.12it/s]


Epoch [2/100], Loss: 0.9626, Test Accuracy: 70.81%


100%|██████████| 391/391 [03:05<00:00,  2.11it/s]


...

Epoch [98/100], Loss: 0.0032, Test Accuracy: 85.50%


100%|██████████| 391/391 [03:06<00:00,  2.10it/s]


Epoch [99/100], Loss: 0.0032, Test Accuracy: 85.58%


100%|██████████| 391/391 [03:06<00:00,  2.10it/s]


Epoch [100/100], Loss: 0.0032, Test Accuracy: 85.44%


In [10]:
# Define the path where you want to save the weights
SAVE_PATH = "savepoints/teacher_resnet101.pth"

# Save the model's state_dict
# This saves only the learned parameters, not the model's architecture
torch.save(model.state_dict(), SAVE_PATH)

print(f"Model weights successfully saved to {SAVE_PATH}")

Model weights successfully saved to savepoints/teacher_resnet101.pth
